In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [3]:
from my_helper import *
from implementations import *

In [4]:
# pre-process
tX = (tX-np.mean(tX, axis=0)) / np.std(tX, axis=0)

In [5]:
# required implementations

max_iters = 50
# least square
# gamma = 0.0000001
# logistic
gamma = 0.000001 
batch_size = 32
lambda_ = 0.1
# initial_w = np.random.rand(tX.shape[1]) * 2 - 1
initial_w = np.zeros(tX.shape[1])
# weights, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)
# weights, loss = least_squares_SGD(y, tX, initial_w, batch_size, max_iters, gamma)
# weights, loss = least_squares(y, tX)
# weights, loss = ridge_regression(y, tX, lambda_)
# weights, loss = logistic_regression_GD(y, tX, initial_w, max_iters, gamma)
# weights, loss = reg_logistic_regression_SGD(y, tX, lambda_, initial_w, batch_size, max_iters, gamma)

In [15]:
# polynomial fit

ratio = 0.9
degree = 10
seed = 6

x_train, y_train, x_val, y_val = split_data(tX, y, ratio, seed)
x_train_poly = build_poly(x_train, degree)
x_val_poly = build_poly(x_val, degree)

lambdas = np.logspace(-5, 2, 30)

for lambda_ in lambdas:
    weights, loss = ridge_regression(y_train, x_train_poly, lambda_)

    y_train_pred = predict_labels(weights, x_train_poly)
    train_acc = cal_acc(y_train, y_train_pred)
    y_val_pred = predict_labels(weights, x_val_poly)
    val_acc = cal_acc(y_val, y_val_pred)

    print("Lambda {labd}, Train accuracy {ta}, Validation Accuracy {va}".format(labd=lambda_, ta=train_acc, va=val_acc))

Lambda 1e-05, Train accuracy 0.8176222222222223, Validation Accuracy 0.8182
Lambda 1.7433288221999873e-05, Train accuracy 0.79532, Validation Accuracy 0.79476
Lambda 3.039195382313195e-05, Train accuracy 0.8160088888888889, Validation Accuracy 0.81592
Lambda 5.2983169062837125e-05, Train accuracy 0.8180711111111111, Validation Accuracy 0.81828
Lambda 9.236708571873866e-05, Train accuracy 0.81744, Validation Accuracy 0.81804
Lambda 0.00016102620275609394, Train accuracy 0.8167377777777778, Validation Accuracy 0.81632
Lambda 0.0002807216203941176, Train accuracy 0.8154577777777777, Validation Accuracy 0.81484
Lambda 0.0004893900918477494, Train accuracy 0.8140177777777777, Validation Accuracy 0.8132
Lambda 0.0008531678524172815, Train accuracy 0.8129466666666667, Validation Accuracy 0.8118
Lambda 0.0014873521072935117, Train accuracy 0.8118355555555555, Validation Accuracy 0.8108
Lambda 0.002592943797404667, Train accuracy 0.8108488888888888, Validation Accuracy 0.80972
Lambda 0.00452035

In [16]:
# best para

ratio = 0.9
degree = 10
lambda_ = 5.2983169062837125e-05
seed = 6

x_poly = build_poly(tX, degree)
weights, loss = ridge_regression(y, x_poly, lambda_)

pred = predict_labels(weights, x_poly)
acc = cal_acc(y, pred)

print(acc)

0.814336


## Generate predictions and save ouput in csv format for submission:

In [18]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
# pre-process
tX_test = (tX_test-np.mean(tX_test, axis=0)) / np.std(tX_test, axis=0)

In [19]:
OUTPUT_PATH = 'predictions.csv' # TODO: fill in desired name of output file for submission
# y_pred = predict_labels(weights, tX_test)
# for polynomial fit
x_test_poly = build_poly(tX_test, degree)
y_pred = predict_labels(weights, x_test_poly)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)